# Testing cosmogan
April 19, 2021

Borrowing pieces of code from : 

- https://github.com/pytorch/tutorials/blob/11569e0db3599ac214b03e01956c2971b02c64ce/beginner_source/dcgan_faces_tutorial.py
- https://github.com/exalearn/epiCorvid/tree/master/cGAN

In [1]:
import os
import random
import logging
import sys

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
#from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel
# import torch.fft

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
# from IPython.display import HTML

import argparse
import time
from datetime import datetime
import glob
import pickle
import yaml
import collections
import socket
import shutil

# # Import modules from other files
# from utils import *
# from spec_loss import *

In [2]:
%matplotlib widget

## Modules

In [3]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)

  
# Generator Code
class View(nn.Module):
    def __init__(self, shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)

def f_get_model(gdict):
    ''' Module to define Generator and Discriminator'''

    if gdict['image_size']==64:

        class Generator(nn.Module):
            def __init__(self, gdict):
                super(Generator, self).__init__()

                ## Define new variables from dict
                keys=['ngpu','nz','nc','ngf','kernel_size','stride','g_padding']
                ngpu, nz,nc,ngf,kernel_size,stride,g_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())

                self.main = nn.Sequential(
                    # nn.ConvTranspose3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                    nn.Linear(nz,nc*ngf*8**3),# 262144
                    nn.BatchNorm3d(nc,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    View(shape=[-1,ngf*8,4,4,4]),
                    nn.ConvTranspose3d(ngf * 8, ngf * 4, kernel_size, stride, g_padding, output_padding=1, bias=False),
                    nn.BatchNorm3d(ngf*4,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    # state size. (ngf*4) x 8 x 8
                    nn.ConvTranspose3d( ngf * 4, ngf * 2, kernel_size, stride, g_padding, 1, bias=False),
                    nn.BatchNorm3d(ngf*2,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    # state size. (ngf*2) x 16 x 16
                    nn.ConvTranspose3d( ngf * 2, ngf, kernel_size, stride, g_padding, 1, bias=False),
                    nn.BatchNorm3d(ngf,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    # state size. (ngf) x 32 x 32
                    nn.ConvTranspose3d( ngf, nc, kernel_size, stride,g_padding, 1, bias=False),
                    nn.Tanh()
                )

            def forward(self, ip):
                return self.main(ip)

        class Discriminator(nn.Module):
            def __init__(self, gdict):
                super(Discriminator, self).__init__()

                ## Define new variables from dict
                keys=['ngpu','nz','nc','ndf','kernel_size','stride','d_padding']
                ngpu, nz,nc,ndf,kernel_size,stride,d_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())        

                self.main = nn.Sequential(
                    # input is (nc) x 64 x 64 x 64
                    # nn.Conv3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                    nn.Conv3d(nc, ndf,kernel_size, stride, d_padding,  bias=True),
                    nn.BatchNorm3d(ndf,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf) x 32 x 32
                    nn.Conv3d(ndf, ndf * 2, kernel_size, stride, d_padding, bias=True),
                    nn.BatchNorm3d(ndf * 2,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*2) x 16 x 16
                    nn.Conv3d(ndf * 2, ndf * 4, kernel_size, stride, d_padding, bias=True),
                    nn.BatchNorm3d(ndf * 4,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*4) x 8 x 8
                    nn.Conv3d(ndf * 4, ndf * 8, kernel_size, stride, d_padding, bias=True),
                    nn.BatchNorm3d(ndf * 8,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*8) x 4 x 4
                    nn.Flatten(),
                    nn.Linear(nc*ndf*8*8*8, 1)
        #             nn.Sigmoid()
                )

            def forward(self, ip):
        #         print(ip.shape)
                results=[ip]
                lst_idx=[]
                for i,submodel in enumerate(self.main.children()):
                    mid_output=submodel(results[-1])
                    results.append(mid_output)
                    ## Select indices in list corresponding to output of Conv layers
                    if submodel.__class__.__name__.startswith('Conv'):
        #                 print(submodel.__class__.__name__)
        #                 print(mid_output.shape)
                        lst_idx.append(i)

                FMloss=True
                if FMloss:
                    ans=[results[1:][i] for i in lst_idx + [-1]]
                else :
                    ans=results[-1]
                return ans

    elif gdict['image_size']==128:

        class Generator(nn.Module):
            def __init__(self, gdict):
                super(Generator, self).__init__()

                ## Define new variables from dict
                keys=['ngpu','nz','nc','ngf','kernel_size','stride','g_padding']
                ngpu, nz,nc,ngf,kernel_size,stride,g_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())

                self.main = nn.Sequential(
                    # nn.ConvTranspose3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                    nn.Linear(nz,nc*ngf*8**3*8),# 262144
                    nn.BatchNorm3d(nc,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    View(shape=[-1,ngf*8,8,8,8]),
                    nn.ConvTranspose3d(ngf * 8, ngf * 4, kernel_size, stride, g_padding, output_padding=1, bias=False),
                    nn.BatchNorm3d(ngf*4,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    # state size. (ngf*4) x 8 x 8
                    nn.ConvTranspose3d( ngf * 4, ngf * 2, kernel_size, stride, g_padding, 1, bias=False),
                    nn.BatchNorm3d(ngf*2,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    # state size. (ngf*2) x 16 x 16
                    nn.ConvTranspose3d( ngf * 2, ngf, kernel_size, stride, g_padding, 1, bias=False),
                    nn.BatchNorm3d(ngf,eps=1e-05, momentum=0.9, affine=True),
                    nn.ReLU(inplace=True),
                    # state size. (ngf) x 32 x 32
                    nn.ConvTranspose3d( ngf, nc, kernel_size, stride,g_padding, 1, bias=False),
                    nn.Tanh()
                )

            def forward(self, ip):
                return self.main(ip)

        class Discriminator(nn.Module):
            def __init__(self, gdict):
                super(Discriminator, self).__init__()

                ## Define new variables from dict
                keys=['ngpu','nz','nc','ndf','kernel_size','stride','d_padding']
                ngpu, nz,nc,ndf,kernel_size,stride,d_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())        

                self.main = nn.Sequential(
                    # input is (nc) x 64 x 64 x 64
                    # nn.Conv3d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
                    nn.Conv3d(nc, ndf,kernel_size, stride, d_padding,  bias=True),
                    nn.BatchNorm3d(ndf,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf) x 32 x 32
                    nn.Conv3d(ndf, ndf * 2, kernel_size, stride, d_padding, bias=True),
                    nn.BatchNorm3d(ndf * 2,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*2) x 16 x 16
                    nn.Conv3d(ndf * 2, ndf * 4, kernel_size, stride, d_padding, bias=True),
                    nn.BatchNorm3d(ndf * 4,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*4) x 8 x 8
                    nn.Conv3d(ndf * 4, ndf * 8, kernel_size, stride, d_padding, bias=True),
                    nn.BatchNorm3d(ndf * 8,eps=1e-05, momentum=0.9, affine=True),
                    nn.LeakyReLU(0.2, inplace=True),
                    # state size. (ndf*8) x 4 x 4
                    nn.Flatten(),
                    nn.Linear(nc*ndf*8*8*8*8, 1)
        #             nn.Sigmoid()
                )

            def forward(self, ip):
                results=[ip]
                lst_idx=[]
                for i,submodel in enumerate(self.main.children()):
                    mid_output=submodel(results[-1])
                    results.append(mid_output)
                    ## Select indices in list corresponding to output of Conv layers
                    if submodel.__class__.__name__.startswith('Conv'):
        #                 print(submodel.__class__.__name__)
        #                 print(mid_output.shape)
                        lst_idx.append(i)

                FMloss=True
                if FMloss:
                    ans=[results[1:][i] for i in lst_idx + [-1]]
                else :
                    ans=results[-1]
                return ans
    
    return Generator, Discriminator


def f_gen_images(gdict,netG,optimizerG,ip_fname,op_loc,op_strg='inf_img_',op_size=500):
    '''Generate images for best saved models
     Arguments: gdict, netG, optimizerG, 
                 ip_fname: name of input file
                op_strg: [string name for output file]
                op_size: Number of images to generate
    '''

    nz,device=gdict['nz'],gdict['device']

    try:# handling cpu vs gpu
        if torch.cuda.is_available(): checkpoint=torch.load(ip_fname)
        else: checkpoint=torch.load(ip_fname,map_location=torch.device('cpu'))
    except Exception as e:
        print(e)
        print("skipping generation of images for ",ip_fname)
        return
    
    ## Load checkpoint
    if gdict['multi-gpu']:
        netG.module.load_state_dict(checkpoint['G_state'])
    else:
        netG.load_state_dict(checkpoint['G_state'])
    
    ## Load other stuff
    iters=checkpoint['iters']
    epoch=checkpoint['epoch']
    optimizerG.load_state_dict(checkpoint['optimizerG_state_dict'])
    
    # Generate batch of latent vectors
    noise = torch.randn(op_size, 1, 1, 1, nz, device=device)
    # Generate fake image batch with G
    netG.eval() ## This is required before running inference
    with torch.no_grad(): ## This is important. fails without it for multi-gpu
        gen = netG(noise)
        gen_images=gen.detach().cpu().numpy()
        print(gen_images.shape)
    
    op_fname='%s_epoch-%s_step-%s.npy'%(op_strg,epoch,iters)
    np.save(op_loc+op_fname,gen_images)

    print("Image saved in ",op_fname)
    
def f_save_checkpoint(gdict,epoch,iters,best_chi1,best_chi2,netG,netD,optimizerG,optimizerD,save_loc):
    ''' Checkpoint model '''
    
    if gdict['multi-gpu']: ## Dataparallel
        torch.save({'epoch':epoch,'iters':iters,'best_chi1':best_chi1,'best_chi2':best_chi2,
                'G_state':netG.module.state_dict(),'D_state':netD.module.state_dict(),'optimizerG_state_dict':optimizerG.state_dict(),
                'optimizerD_state_dict':optimizerD.state_dict()}, save_loc) 
    else :
        torch.save({'epoch':epoch,'iters':iters,'best_chi1':best_chi1,'best_chi2':best_chi2,
                'G_state':netG.state_dict(),'D_state':netD.state_dict(),'optimizerG_state_dict':optimizerG.state_dict(),
                'optimizerD_state_dict':optimizerD.state_dict()}, save_loc)
    
def f_load_checkpoint(ip_fname,netG,netD,optimizerG,optimizerD,gdict):
    ''' Load saved checkpoint
    Also loads step, epoch, best_chi1, best_chi2'''
    
    print("torch device",torch.device('cuda',torch.cuda.current_device()))
            
    try:
        checkpoint=torch.load(ip_fname,map_location=torch.device('cuda',torch.cuda.current_device()))
    except Exception as e:
        print("Error loading saved checkpoint",ip_fname)
        print(e)
        raise SystemError
    
    ## Load checkpoint
    if gdict['multi-gpu']:
        netG.module.load_state_dict(checkpoint['G_state'])
        netD.module.load_state_dict(checkpoint['D_state'])
    else:
        netG.load_state_dict(checkpoint['G_state'])
        netD.load_state_dict(checkpoint['D_state'])
    
    optimizerD.load_state_dict(checkpoint['optimizerD_state_dict'])
    optimizerG.load_state_dict(checkpoint['optimizerG_state_dict'])
    
    iters=checkpoint['iters']
    epoch=checkpoint['epoch']
    best_chi1=checkpoint['best_chi1']
    best_chi2=checkpoint['best_chi2']

    netG.train()
    netD.train()
    
    return iters,epoch,best_chi1,best_chi2,netD,optimizerD,netG,optimizerG


In [4]:
####################
### Pytorch code ###
####################

def f_get_rad(img):
    ''' Get the radial tensor for use in f_torch_get_azimuthalAverage '''
    
    height,width,depth=img.shape[-3:]
    # Create a grid of points with x and y and z coordinates
    z,y,x = np.indices([height,width,depth])
    
    center=[]
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0, (z.max()-z.min())/2.0])

    # Get the radial coordinate for every grid point. Array has the shape of image
    r= torch.tensor(np.sqrt((x-center[0])**2 + (y-center[1])**2 + (z-center[2])**2))
        
    # Get sorted radii
    ind = torch.argsort(torch.reshape(r, (-1,)))

    return r.detach(),ind.detach()


def f_torch_get_azimuthalAverage(image,r,ind):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    source: https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/
    """
    
#     height, width = image.shape
#     # Create a grid of points with x and y coordinates
#     y, x = np.indices([height,width])

#     if not center:
#         center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])

#     # Get the radial coordinate for every grid point. Array has the shape of image
#     r = torch.tensor(np.hypot(x - center[0], y - center[1]))

#     # Get sorted radii
#     ind = torch.argsort(torch.reshape(r, (-1,)))

    r_sorted = torch.gather(torch.reshape(r, ( -1,)),0, ind)
    i_sorted = torch.gather(torch.reshape(image, ( -1,)),0, ind)
    
    # Get the integer part of the radii (bin size = 1)
    r_int=r_sorted.to(torch.int32)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = torch.reshape(torch.where(deltar)[0], (-1,))    # location of changes in radius
    nr = (rind[1:] - rind[:-1]).type(torch.float)       # number of radius bin

    # Cumulative sum to figure out sums for each radius bin
    
    csum = torch.cumsum(i_sorted, axis=-1)
    tbin = torch.gather(csum, 0, rind[1:]) - torch.gather(csum, 0, rind[:-1])
    radial_prof = tbin / nr

    return radial_prof

def f_torch_fftshift(real, imag):
    for dim in range(0, len(real.size())):
        real = torch.roll(real, dims=dim, shifts=real.size(dim)//2)
        imag = torch.roll(imag, dims=dim, shifts=imag.size(dim)//2)
    return real, imag

def f_torch_compute_spectrum(arr,r,ind):
    
    GLOBAL_MEAN=1.0
    arr=(arr-GLOBAL_MEAN)/(GLOBAL_MEAN)
    
    y1=torch.rfft(arr,signal_ndim=3,onesided=False)
    real,imag=f_torch_fftshift(y1[:,:,:,0],y1[:,:,:,1])    ## last index is real/imag part  ## Mod for 3D
    
#     # For pytorch 1.8
#     y1=torch.fft.fftn(arr,dim=(-3,-2,-1))
#     real,imag=f_torch_fftshift(y1.real,y1.imag)    
    
    y2=real**2+imag**2     ## Absolute value of each complex number
    z1=f_torch_get_azimuthalAverage(y2,r,ind)     ## Compute radial profile
    return z1

def f_torch_compute_batch_spectrum(arr,r,ind):
    
    batch_pk=torch.stack([f_torch_compute_spectrum(i,r,ind) for i in arr])
    
    return batch_pk

def f_torch_image_spectrum(x,num_channels,r,ind):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    mean=[[] for i in range(num_channels)]    
    var=[[] for i in range(num_channels)] 

    for i in range(num_channels):
        arr=x[:,i,:,:,:] # Mod for 3D
        batch_pk=f_torch_compute_batch_spectrum(arr,r,ind)
        mean[i]=torch.mean(batch_pk,axis=0)
#         var[i]=torch.std(batch_pk,axis=0)/np.sqrt(batch_pk.shape[0])
#         var[i]=torch.std(batch_pk,axis=0)
        var[i]=torch.var(batch_pk,axis=0)
    
    mean=torch.stack(mean)
    var=torch.stack(var)
        
    if (torch.isnan(mean).any() or torch.isnan(var).any()):
        print("Nans in spectrum",mean,var)
        if torch.isnan(x).any():
            print("Nans in Input image")

    return mean,var

def f_compute_hist(data,bins):
    
    try: 
        hist_data=torch.histc(data,bins=bins)
        ## A kind of normalization of histograms: divide by total sum
        hist_data=(hist_data*bins)/torch.sum(hist_data)
    except Exception as e:
        print(e)
        hist_data=torch.zeros(bins)

    return hist_data

### Losses 
def loss_spectrum(spec_mean,spec_mean_ref,spec_var,spec_var_ref,image_size,lambda_spec_mean,lambda_spec_var):
    ''' Loss function for the spectrum : mean + variance 
    Log(sum( batch value - expect value) ^ 2 )) '''
    
    if (torch.isnan(spec_mean).any() or torch.isnan(spec_var).any()):
        ans=torch.tensor(float("inf"))
        return ans
    
    idx=int(image_size/2) ### For the spectrum, use only N/2 indices for loss calc.
    ### Warning: the first index is the channel number.For multiple channels, you are averaging over them, which is fine.
        
    loss_mean=torch.log(torch.mean(torch.pow(spec_mean[:,:idx]-spec_mean_ref[:,:idx],2)))
    loss_var=torch.log(torch.mean(torch.pow(spec_var[:,:idx]-spec_var_ref[:,:idx],2)))
    
    ans=lambda_spec_mean*loss_mean+lambda_spec_var*loss_var
    
    if (torch.isnan(ans).any()) :    
        print("loss spec mean %s, loss spec var %s"%(loss_mean,loss_var))
        print("spec mean %s, ref %s"%(spec_mean, spec_mean_ref))
        print("spec var %s, ref %s"%(spec_var, spec_var_ref))
#         raise SystemExit
        
    return ans
    
def loss_hist(hist_sample,hist_ref):
    
    lambda1=1.0
    return lambda1*torch.log(torch.mean(torch.pow(hist_sample-hist_ref,2)))

def f_FM_loss(real_output,fake_output,lambda_fm,gdict):
    '''
    Module to implement Feature-Matching loss. Reads all but last elements of Discriminator ouput
    '''
    FM=torch.Tensor([0.0]).to(gdict['device'])
    for i,j in zip(real_output[:-1],fake_output[:-1]):
#         print(i.shape,j.shape)
        real_mean=torch.mean(i)
        fake_mean=torch.mean(j)
#         print(real_mean,fake_mean)
        FM=FM.clone()+torch.sum(torch.square(real_mean-fake_mean))
    return lambda_fm*FM

def f_gp_loss(grads,l=1.0):
    '''
    Module to implement gradient penalty loss.
    '''
    loss=torch.mean(torch.sum(torch.square(grads),dim=[1,2,3]))
    return l*loss

## Train loop

In [15]:
### Train code ###
def f_train_loop(gan_model,Dset,metrics_df,gdict,fixed_noise):
    ''' Train epochs '''
    ## Define new variables from dict
    keys=['image_size','start_epoch','epochs','iters','best_chi1','best_chi2','save_dir','device','flip_prob','nz','batch_size','bns']
    image_size,start_epoch,epochs,iters,best_chi1,best_chi2,save_dir,device,flip_prob,nz,batchsize,bns=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())
    
    for epoch in range(start_epoch,epochs):
        t_epoch_start=time.time()
        for count, data in enumerate(Dset.train_dataloader):

            ####### Train GAN ########
            gan_model.netG.train(); gan_model.netD.train();  ### Need to add these after inference and before training

            tme1=time.time()
            ### Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            gan_model.netD.zero_grad()

            real_cpu = data[0].to(device)
            real_cpu.requires_grad=True
            b_size = real_cpu.size(0)
            real_label = torch.full((b_size,), 1, device=device,dtype=float)
            fake_label = torch.full((b_size,), 0, device=device,dtype=float)
            g_label = torch.full((b_size,), 1, device=device,dtype=float) ## No flipping for Generator labels
            # Flip labels with probability flip_prob
            for idx in np.random.choice(np.arange(b_size),size=int(np.ceil(b_size*flip_prob))):
                real_label[idx]=0; fake_label[idx]=1

            # Generate fake image batch with G
            noise = torch.randn(b_size, 1, 1, 1, nz, device=device) ### Mod for 3D
            fake = gan_model.netG(noise)            

            # Forward pass real batch through D
            real_output = gan_model.netD(real_cpu)
            errD_real = gan_model.criterion(real_output[-1].view(-1), real_label.float())
            errD_real.backward(retain_graph=True)
            D_x = real_output[-1].mean().item()

            # Forward pass fake batch through D
            fake_output = gan_model.netD(fake.detach())   # The detach is important
            errD_fake = gan_model.criterion(fake_output[-1].view(-1), fake_label.float())
            errD_fake.backward(retain_graph=True)
            D_G_z1 = fake_output[-1].mean().item()
            
            errD = errD_real + errD_fake 

            if gdict['lambda_gp']: ## Add gradient - penalty loss
                grads=torch.autograd.grad(outputs=real_output[-1],inputs=real_cpu,grad_outputs=torch.ones_like(real_output[-1]),allow_unused=False,create_graph=True)[0]
                gp_loss=f_gp_loss(grads,gdict['lambda_gp'])
                gp_loss.backward(retain_graph=True)
                errD = errD + gp_loss
            else:
                gp_loss=torch.Tensor([np.nan])
                
            if gdict['grad_clip']:
                nn.utils.clip_grad_norm_(gan_model.netD.parameters(),gdict['grad_clip'])

            gan_model.optimizerD.step()
            lr_d=gan_model.optimizerD.param_groups[0]['lr']
            gan_model.schedulerD.step()
            
# dict_keys(['train_data_loader', 'r', 'ind', 'train_spec_mean', 'train_spec_var', 'train_hist', 'val_spec_mean', 'val_spec_var', 'val_hist'])

            ###Update G network: maximize log(D(G(z)))
            gan_model.netG.zero_grad()
            output = gan_model.netD(fake)
            errG_adv = gan_model.criterion(output[-1].view(-1), g_label.float())
#             errG_adv.backward(retain_graph=True)
            # Histogram pixel intensity loss
            hist_gen=f_compute_hist(fake,bins=bns)
            hist_loss=loss_hist(hist_gen,Dset.train_hist.to(device))

            # Add spectral loss
            mean,var=f_torch_image_spectrum(f_invtransform(fake),1,Dset.r.to(device),Dset.ind.to(device))
            spec_loss=loss_spectrum(mean,Dset.train_spec_mean.to(device),var,Dset.train_spec_var.to(device),image_size,gdict['lambda_spec_mean'],gdict['lambda_spec_var'])

            errG=errG_adv
            if gdict['lambda_spec_mean']: 
#                 spec_loss.backward(retain_graph=True)
                errG = errG+ spec_loss 
            if gdict['lambda_fm']:## Add feature matching loss
                fm_loss=f_FM_loss([i.detach() for i in real_output],output,gdict['lambda_fm'],gdict)
#                 fm_loss.backward(retain_graph=True)
                errG= errG+ fm_loss
            else: 
                fm_loss=torch.Tensor([np.nan])

            if torch.isnan(errG).any():
                logging.info(errG)
                raise SystemError
            
            # Calculate gradients for G
            errG.backward()
            D_G_z2 = output[-1].mean().item()
            
            ### Implement Gradient clipping
            if gdict['grad_clip']:
                nn.utils.clip_grad_norm_(gan_model.netG.parameters(),gdict['grad_clip'])
            
            gan_model.optimizerG.step()
            lr_g=gan_model.optimizerG.param_groups[0]['lr']
            gan_model.schedulerG.step()
            
            tme2=time.time()
            ####### Store metrics ########
            # Output training stats
            if gdict['world_rank']==0:
                if ((count % gdict['checkpoint_size'] == 0)):
                    logging.info('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_adv: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                          % (epoch, epochs, count, len(Dset.train_dataloader), errD.item(), errG_adv.item(),errG.item(), D_x, D_G_z1, D_G_z2)),
                    logging.info("Spec loss: %s,\t hist loss: %s"%(spec_loss.item(),hist_loss.item())),
                    logging.info("Training time for step %s : %s"%(iters, tme2-tme1))

                # Save metrics
                cols=['step','epoch','Dreal','Dfake','Dfull','G_adv','G_full','spec_loss','hist_loss','fm_loss','gp_loss','D(x)','D_G_z1','D_G_z2','lr_d','lr_g','time']
                vals=[iters,epoch,errD_real.item(),errD_fake.item(),errD.item(),errG_adv.item(),errG.item(),spec_loss.item(),hist_loss.item(),fm_loss.item(),gp_loss.item(),D_x,D_G_z1,D_G_z2,lr_d,lr_g,tme2-tme1]
                for col,val in zip(cols,vals):  metrics_df.loc[iters,col]=val

                ### Checkpoint the best model
                checkpoint=True
                iters += 1  ### Model has been updated, so update iters before saving metrics and model.

                ### Compute validation metrics for updated model
                gan_model.netG.eval()
                with torch.no_grad():
                    fake = gan_model.netG(fixed_noise)
                    hist_gen=f_compute_hist(fake,bins=bns)
                    hist_chi=loss_hist(hist_gen,Dset.val_hist.to(device))
                    mean,var=f_torch_image_spectrum(f_invtransform(fake),1,Dset.r.to(device),Dset.ind.to(device))
                    spec_chi=loss_spectrum(mean,Dset.val_spec_mean.to(device),var,Dset.val_spec_var.to(device),image_size,gdict['lambda_spec_mean'],gdict['lambda_spec_var'])

                # Storing chi for next step
                for col,val in zip(['spec_chi','hist_chi'],[spec_chi.item(),hist_chi.item()]):  metrics_df.loc[iters,col]=val            

                # Checkpoint model for continuing run
                if count == len(Dset.train_dataloader)-1: ## Check point at last step of epoch
                    f_save_checkpoint(gdict,epoch,iters,best_chi1,best_chi2,gan_model.netG,gan_model.netD,gan_model.optimizerG,gan_model.optimizerD,save_loc=save_dir+'/models/checkpoint_last.tar')  

                if (checkpoint and (epoch > 1)): # Choose best models by metric
                    if hist_chi< best_chi1:
                        f_save_checkpoint(gdict,epoch,iters,best_chi1,best_chi2,gan_model.netG,gan_model.netD,gan_model.optimizerG,gan_model.optimizerD,save_loc=save_dir+'/models/checkpoint_best_hist.tar')
                        best_chi1=hist_chi.item()
                        logging.info("Saving best hist model at epoch %s, step %s."%(epoch,iters))

                    if  spec_chi< best_chi2:
                        f_save_checkpoint(gdict,epoch,iters,best_chi1,best_chi2,gan_model.netG,gan_model.netD,gan_model.optimizerG,gan_model.optimizerD,save_loc=save_dir+'/models/checkpoint_best_spec.tar')
                        best_chi2=spec_chi.item()
                        logging.info("Saving best spec model at epoch %s, step %s"%(epoch,iters))

#                    if (iters in gdict['save_steps_list']) :
                    if ((gdict['save_steps_list']=='all') and (iters % gdict['checkpoint_size'] == 0)):
                        f_save_checkpoint(gdict,epoch,iters,best_chi1,best_chi2,gan_model.netG,gan_model.netD,gan_model.optimizerG,gan_model.optimizerD,save_loc=save_dir+'/models/checkpoint_{0}.tar'.format(iters))
                        logging.info("Saving given-step at epoch %s, step %s."%(epoch,iters))

                # Save G's output on fixed_noise
                if ((iters % gdict['checkpoint_size'] == 0) or ((epoch == epochs-1) and (count == len(Dset.train_dataloader)-1))):
                    gan_model.netG.eval()
                    with torch.no_grad():
                        fake = gan_model.netG(fixed_noise).detach().cpu()
                        img_arr=np.array(fake)
                        fname='gen_img_epoch-%s_step-%s'%(epoch,iters)
                        np.save(save_dir+'/images/'+fname,img_arr)
        
        t_epoch_end=time.time()
        if gdict['world_rank']==0:
            logging.info("Time taken for epoch %s, count %s: %s for rank %s"%(epoch,count,t_epoch_end-t_epoch_start,gdict['world_rank']))
            # Save Metrics to file after each epoch
            metrics_df.to_pickle(save_dir+'/df_metrics.pkle')
            logging.info("best chis: {0}, {1}".format(best_chi1,best_chi2))



## Start

In [28]:
### Setup modules ###
def f_manual_add_argparse():
    ''' use only in jpt notebook'''
    args=argparse.Namespace()
    args.config='config_3d_gan.yaml'
    args.mode='fresh'
    args.local_rank=0
    args.facility='cori'
    args.distributed=False

#     args.mode='continue'
    
    return args

def f_parse_args():
    """Parse command line arguments.Only for .py file"""
    parser = argparse.ArgumentParser(description="Run script to train GAN using pytorch", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    add_arg = parser.add_argument
    
    add_arg('--config','-cfile',  type=str, default='config_3d_Cgan.yaml', help='Name of config file')
    add_arg('--mode','-m',  type=str, choices=['fresh','continue','fresh_load'],default='fresh', help='Whether to start fresh run or continue previous run or fresh run loading a config file.')
    add_arg("--local_rank", default=0, type=int,help='Local rank of GPU on node. Using for pytorch DDP. ')
    add_arg("--facility", default='cori', choices=['cori','summit'],type=str,help='Facility: cori or summit ')
    add_arg("--ddp", dest='distributed' ,default=False,action='store_true',help='use Distributed DataParallel for Pytorch or DataParallel')
    
    return parser.parse_args()

def try_barrier(rank):
    """
    Used in Distributed data parallel
    Attempt a barrier but ignore any exceptions
    """
    print('BAR %d'%rank)
    try:
        dist.barrier()
    except:
        pass

def f_init_gdict(args,gdict):
    ''' Create global dictionary gdict from args and config file'''
    
    ## read config file
    config_file=args.config
    with open(config_file) as f:
        config_dict= yaml.load(f, Loader=yaml.SafeLoader)
        
    gdict=config_dict['parameters']

    args_dict=vars(args)
    ## Add args variables to gdict
    for key in args_dict.keys():
        gdict[key]=args_dict[key]

    if gdict['distributed']: 
        assert not gdict['lambda_gp'],"GP couplings is %s. Cannot use Gradient penalty loss in pytorch DDP"%(gdict['lambda_gp'])
    else : print("Not using DDP")
    return gdict


def f_get_img_samples(ip_arr,rank=0,num_ranks=1):
    '''
    Module to get part of the numpy image file
    '''
    
    data_size=ip_arr.shape[0]
    size=data_size//num_ranks
    
    if gdict['batch_size']>size:
        print("Caution: batchsize %s is greater than samples per GPU %s"%(gdict['batch_size'],size))
        raise SystemExit
        
    ### Get a set of random indices from numpy array
    random=False
    if random:
        idxs=np.arange(ip_arr.shape[0])
        np.random.shuffle(idxs)
        rnd_idxs=idxs[rank*(size):(rank+1)*size]
        arr=ip_arr[rnd_idxs].copy()
        
    else: arr=ip_arr[rank*(size):(rank+1)*size].copy()
    
    return arr

def f_setup(gdict,metrics_df,log):
    ''' 
    Set up directories, Initialize random seeds, add GPU info, add logging info.
    '''
    
    torch.backends.cudnn.benchmark=True
#     torch.autograd.set_detect_anomaly(True)

    ## New additions. Code taken from Jan B.
    os.environ['MASTER_PORT'] = "8885"

    if gdict['facility']=='summit':
        get_master = "echo $(cat {} | sort | uniq | grep -v batch | grep -v login | head -1)".format(os.environ['LSB_DJOB_HOSTFILE'])
        os.environ['MASTER_ADDR'] = str(subprocess.check_output(get_master, shell=True))[2:-3]
        os.environ['WORLD_SIZE'] = os.environ['OMPI_COMM_WORLD_SIZE']
        os.environ['RANK'] = os.environ['OMPI_COMM_WORLD_RANK']
        gdict['local_rank'] = int(os.environ['OMPI_COMM_WORLD_LOCAL_RANK'])
    else:
        if gdict['distributed']:
            os.environ['WORLD_SIZE'] = os.environ['SLURM_NTASKS']
            os.environ['RANK'] = os.environ['SLURM_PROCID']
            gdict['local_rank'] = int(os.environ['SLURM_LOCALID'])

    ## Special declarations
    gdict['ngpu']=torch.cuda.device_count()
    gdict['device']=torch.device("cuda" if (torch.cuda.is_available()) else "cpu")
    gdict['multi-gpu']=True if (gdict['device'].type == 'cuda') and (gdict['ngpu'] > 1) else False 
    
    ########################
    ###### Set up Distributed Data parallel ######
    if gdict['distributed']:
#         gdict['local_rank']=args.local_rank  ## This is needed when using pytorch -m torch.distributed.launch
        gdict['world_size']=int(os.environ['WORLD_SIZE'])
        torch.cuda.set_device(gdict['local_rank']) ## Very important
        dist.init_process_group(backend='nccl', init_method="env://")  
        gdict['world_rank']= dist.get_rank()
        
        device = torch.cuda.current_device()
        logging.info("World size %s, world rank %s, local rank %s device %s, hostname %s, GPUs on node %s\n"%(gdict['world_size'],gdict['world_rank'],gdict['local_rank'],device,socket.gethostname(),gdict['ngpu']))
        
        # Divide batch size by number of GPUs
#         gdict['batch_size']=gdict['batch_size']//gdict['world_size']
    else:
        gdict['world_size'],gdict['world_rank'],gdict['local_rank']=1,0,0
    
    ########################
    ###### Set up directories #######
    ### sync up so that time is the same for each GPU for DDP
    if gdict['mode'] in ['fresh','fresh_load']:
        ### Create prefix for foldername      
        if gdict['world_rank']==0: ### For rank=0, create directory name string and make directories
            dt_strg=datetime.now().strftime('%Y%m%d_%H%M%S') ## time format
            dt_lst=[int(i) for i in dt_strg.split('_')] # List storing day and time            
            dt_tnsr=torch.LongTensor(dt_lst).to(gdict['device'])  ## Create list to pass to other GPUs 

        else: dt_tnsr=torch.Tensor([0,0]).long().to(gdict['device'])
        ### Pass directory name to other ranks
        if gdict['distributed']: dist.broadcast(dt_tnsr, src=0)

        gdict['save_dir']=gdict['op_loc']+str(int(dt_tnsr[0]))+'_'+str(int(dt_tnsr[1]))+'_'+gdict['run_suffix']
        
        if gdict['world_rank']==0: # Create directories for rank 0
            ### Create directories
            if not os.path.exists(gdict['save_dir']):
                os.makedirs(gdict['save_dir']+'/models')
                os.makedirs(gdict['save_dir']+'/images')
                shutil.copy(gdict['config'],gdict['save_dir'])    
    
    elif gdict['mode']=='continue': ## For checkpointed runs
        gdict['save_dir']=gdict['ip_fldr']
        ### Read loss data
        metrics_df=pd.read_pickle(gdict['save_dir']+'/df_metrics.pkle').astype(np.float64)
   
    ########################
    ### Initialize random seed
    
    manualSeed = np.random.randint(1, 10000) if gdict['seed']=='random' else int(gdict['seed'])
#     print("Seed",manualSeed,gdict['world_rank'])
    random.seed(manualSeed)
    np.random.seed(manualSeed)
    torch.manual_seed(manualSeed)
    torch.cuda.manual_seed_all(manualSeed)
    
    if gdict['deterministic']:
        logging.info("Running with deterministic sequence. Performance will be slower")
        torch.backends.cudnn.deterministic=True
#         torch.backends.cudnn.enabled = False
        torch.backends.cudnn.benchmark = False        
    
    ########################
    if log:
        ### Write all logging.info statements to stdout and log file
        logfile=gdict['save_dir']+'/log.log'
        if gdict['world_rank']==0:
            logging.basicConfig(level=logging.DEBUG, filename=logfile, filemode="a+", format="%(asctime)-15s %(levelname)-8s %(message)s")

            Lg = logging.getLogger()
            Lg.setLevel(logging.DEBUG)
            lg_handler_file = logging.FileHandler(logfile)
            lg_handler_stdout = logging.StreamHandler(sys.stdout)
            Lg.addHandler(lg_handler_file)
            Lg.addHandler(lg_handler_stdout)

            logging.info('Args: {0}'.format(args))
            logging.info('Start: %s'%(datetime.now().strftime('%Y-%m-%d  %H:%M:%S')))
        
        if gdict['distributed']:  try_barrier(gdict['world_rank'])

        if gdict['world_rank']!=0:
                logging.basicConfig(level=logging.DEBUG, filename=logfile, filemode="a+", format="%(asctime)-15s %(levelname)-8s %(message)s")

    return metrics_df

class Dataset:
    def __init__(self,gdict):
        '''
        Load training dataset and compute spectrum and histogram for a small batch of training and validation dataset.
        '''
        ## Load training dataset
        t0a=time.time()
        img=np.load(gdict['ip_fname'],mmap_mode='r')[:gdict['num_imgs']]
    #     print("Shape of input file",img.shape)
        img=f_get_img_samples(img,gdict['world_rank'],gdict['world_size'])  

        t_img=torch.from_numpy(img)
        dataset=TensorDataset(t_img)
        self.train_dataloader=DataLoader(dataset,batch_size=gdict['batch_size'],shuffle=True,num_workers=0,drop_last=True)
        logging.info("Size of dataset for GPU %s : %s"%(gdict['world_rank'],len(self.train_dataloader.dataset)))

        t0b=time.time()
        logging.info("Time for creating dataloader",t0b-t0a,gdict['world_rank'])
        
        # Precompute spectrum and histogram for small training and validation data for computing losses
        with torch.no_grad():
            val_img=np.load(gdict['ip_fname'],mmap_mode='r')[-100:].copy()
            t_val_img=torch.from_numpy(val_img).to(gdict['device'])
            # Precompute radial coordinates
            r,ind=f_get_rad(val_img)
            self.r,self.ind=r.to(gdict['device']),ind.to(gdict['device'])

            # Compute
            self.train_spec_mean,self.train_spec_var=f_torch_image_spectrum(f_invtransform(t_val_img),1,self.r,self.ind)
            self.train_hist=f_compute_hist(t_val_img,bins=gdict['bns'])
            
            # Repeat for validation dataset
            val_img=np.load(gdict['ip_fname'],mmap_mode='r')[-200:-100].copy()
            t_val_img=torch.from_numpy(val_img).to(gdict['device'])
            
            # Compute
            self.val_spec_mean,self.val_spec_var=f_torch_image_spectrum(f_invtransform(t_val_img),1,self.r,self.ind)
            self.val_hist=f_compute_hist(t_val_img,bins=gdict['bns'])
            del val_img; del t_val_img; del img; del t_img;

class GAN_model():
    def __init__(self,gdict,print_model=False):
    
        def weights_init(m):
            '''custom weights initialization called on netG and netD '''
            classname = m.__class__.__name__
            if classname.find('Conv') != -1:
                nn.init.normal_(m.weight.data, 0.0, 0.02)
            elif classname.find('BatchNorm') != -1:
                nn.init.normal_(m.weight.data, 1.0, 0.02)
                nn.init.constant_(m.bias.data, 0)
        
        ## Choose model
        Generator, Discriminator=f_get_model(gdict) ## Mod for cGAN
        
        # Create Generator
        self.netG = Generator(gdict).to(gdict['device'])
        self.netG.apply(weights_init)
        # Create Discriminator
        self.netD = Discriminator(gdict).to(gdict['device'])
        self.netD.apply(weights_init)

        if print_model:
            if gdict['world_rank']==0:
                print(self.netG)
            #     summary(netG,(1,1,64))
                print(self.netD)
            #     summary(netD,(1,128,128))
                print("Number of GPUs used %s"%(gdict['ngpu']))

        if (gdict['multi-gpu']):
            if not gdict['distributed']:
                self.netG = nn.DataParallel(self.netG, list(range(gdict['ngpu'])))
                self.netD = nn.DataParallel(self.netD, list(range(gdict['ngpu'])))
            else:
                self.netG=DistributedDataParallel(self.netG,device_ids=[gdict['local_rank']],output_device=[gdict['local_rank']])
                self.netD=DistributedDataParallel(self.netD,device_ids=[gdict['local_rank']],output_device=[gdict['local_rank']])

        #### Initialize networks ####
        # self.criterion = nn.BCELoss()
        self.criterion = nn.BCEWithLogitsLoss()

        self.optimizerD = optim.Adam(self.netD.parameters(), lr=gdict['learn_rate_d'], betas=(gdict['beta1'], 0.999),eps=1e-7)
        self.optimizerG = optim.Adam(self.netG.parameters(), lr=gdict['learn_rate_g'], betas=(gdict['beta1'], 0.999),eps=1e-7)
        
        if gdict['distributed']:  try_barrier(gdict['world_rank'])

        if gdict['mode']=='fresh':
            iters,start_epoch,best_chi1,best_chi2=0,0,1e10,1e10 
            
        elif gdict['mode']=='continue':
            iters,start_epoch,best_chi1,best_chi2,self.netD,self.optimizerD,self.netG,self.optimizerG=f_load_checkpoint(gdict['save_dir']+'/models/checkpoint_last.tar',\
                                                                                                                        self.netG,self.netD,self.optimizerG,self.optimizerD,gdict) 
            if gdict['world_rank']==0: logging.info("\nContinuing existing run. Loading checkpoint with epoch {0} and step {1}\n".format(start_epoch,iters))
            if gdict['distributed']:  try_barrier(gdict['world_rank'])
            start_epoch+=1  ## Start with the next epoch 
        
        elif gdict['mode']=='fresh_load':
            iters,start_epoch,best_chi1,best_chi2,self.netD,self.optimizerD,self.netG,self.optimizerG=f_load_checkpoint(gdict['chkpt_file'],\
                                                                                                                        self.netG,self.netD,self.optimizerG,self.optimizerD,gdict) 
            if gdict['world_rank']==0: logging.info("Fresh run loading checkpoint file {0}".format(gdict['chkpt_file']))
#             if gdict['distributed']:  try_barrier(gdict['world_rank'])
            iters,start_epoch,best_chi1,best_chi2=0,0,1e10,1e10 
        
        ## Add to gdict
        for key,val in zip(['best_chi1','best_chi2','iters','start_epoch'],[best_chi1,best_chi2,iters,start_epoch]): gdict[key]=val
        
        ## Set up learn rate scheduler
        lr_stepsize=int((gdict['num_imgs'])/(gdict['batch_size']*gdict['world_size'])) # convert epoch number to step 
        lr_d_epochs=[i*lr_stepsize for i in gdict['lr_d_epochs']] 
        lr_g_epochs=[i*lr_stepsize for i in gdict['lr_g_epochs']]
        self.schedulerD = optim.lr_scheduler.MultiStepLR(self.optimizerD, milestones=lr_d_epochs,gamma=gdict['lr_d_gamma'])
        self.schedulerG = optim.lr_scheduler.MultiStepLR(self.optimizerG, milestones=lr_g_epochs,gamma=gdict['lr_g_gamma'])


## Main

In [29]:
#########################
### Main code #######
#########################

if __name__=="__main__":
    jpt=False
    jpt=True ##(different for jupyter notebook)
    t0=time.time()
    t0=time.time()
    args=f_parse_args() if not jpt else f_manual_add_argparse()

    #################################
    ### Set up global dictionary###
    gdict={}
    gdict=f_init_gdict(args,gdict)
#     gdict['num_imgs']=200

    if jpt: ## override for jpt nbks
        gdict['num_imgs']=400
        gdict['run_suffix']='nb_test'
        
    ### Set up metrics dataframe
    cols=['step','epoch','Dreal','Dfake','Dfull','G_adv','G_full','spec_loss','hist_loss','spec_chi','hist_chi','gp_loss','fm_loss','D(x)','D_G_z1','D_G_z2','time']
    metrics_df=pd.DataFrame(columns=cols)
    
    # Setup
    metrics_df=f_setup(gdict,metrics_df,log=(not jpt))
    
    ## Build GAN
    gan_model=GAN_model(gdict,False)
    fixed_noise = torch.randn(gdict['op_size'], 1, 1, 1, gdict['nz'], device=gdict['device']) #Latent vectors to view G progress    # Mod for 3D

    if gdict['distributed']:  try_barrier(gdict['world_rank'])

    ## Load data and precompute
    Dset=Dataset(gdict)
    
    #################################
    ########## Train loop and save metrics and images ######
    if gdict['distributed']:  try_barrier(gdict['world_rank'])

    if gdict['world_rank']==0: 
        logging.info(gdict)
        logging.info("Starting Training Loop...")
        
    f_train_loop(gan_model,Dset,metrics_df,gdict,fixed_noise)
    
    if gdict['world_rank']==0: ## Generate images for best saved models ######
        op_loc=gdict['save_dir']+'/images/'
        ip_fname=gdict['save_dir']+'/models/checkpoint_best_spec.tar'
        f_gen_images(gdict,gan_model.netG,gan_model.optimizerG,ip_fname,op_loc,op_strg='best_spec',op_size=32)
        ip_fname=gdict['save_dir']+'/models/checkpoint_best_hist.tar'
        f_gen_images(gdict,gan_model.netG,gan_model.optimizerG,ip_fname,op_loc,op_strg='best_hist',op_size=32)
    
    tf=time.time()
    logging.info("Total time %s"%(tf-t0))
    logging.info('End: %s'%(datetime.now().strftime('%Y-%m-%d  %H:%M:%S')))


Not using DDP
Nans in spectrum tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan]], device='cuda:0') tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan]], device='cuda:0')
Nans in spectrum tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan]], device='cuda:0', grad_fn=<StackBackw

In [30]:
# metrics_df.plot('step','time')
metrics_df

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
0,0,0,0.840994,0.715251,1.556245,22.584732,33.839798,11.255068,1.612284,NaN,NaN,NaN,NaN,-0.07203,-0.208424,-17.184401,0.103338,0.001000,0.001
1,1,0,1.299035,2.241939,3.540974,28.161259,39.285881,11.12462,1.21745,11.183544,0.758419,NaN,NaN,6.140594,2.899389,-28.161259,0.103051,0.001000,0.001
2,2,0,10.559894,5.898785,16.458679,4.830589,15.81959,10.989,1.072545,inf,1.118342,NaN,NaN,-13.925358,-22.697987,-4.822292,0.103292,0.001000,0.001
3,3,0,0.49855,0.666836,1.165386,0.0,inf,inf,1.048596,inf,0.91501,NaN,NaN,0.014235,-1.888329,23.090822,0.102724,0.001000,0.001
4,4,0,3.799689,17.408419,21.208107,0.016709,inf,inf,1.064357,inf,0.919925,NaN,NaN,16.109097,23.190767,4.118836,0.101315,0.001000,0.001
5,5,0,1.962665,3.645362,5.608027,1.896615,inf,inf,1.058407,inf,0.995578,NaN,NaN,9.117305,4.861509,-1.73393,0.101913,0.000250,0.001
6,6,0,0.639535,0.791798,1.431333,7.104887,inf,inf,1.111167,inf,1.144822,NaN,NaN,3.23758,0.376904,-7.104041,0.101668,0.000250,0.001
7,7,0,2.044566,0.753196,2.797762,2.649329,inf,inf,1.279575,inf,1.351185,NaN,NaN,-3.029798,-3.322231,-2.571097,0.101727,0.000250,0.001
8,8,0,0.339161,3.919532,4.258693,5.264164,inf,inf,1.356508,inf,1.388227,NaN,NaN,2.735115,5.148591,-5.258847,0.101463,0.000250,0.001
9,9,0,2.161517,0.857188,3.018704,8.289993,inf,inf,1.382153,inf,1.442253,NaN,NaN,-0.884884,0.559244,-8.289726,0.101667,0.000250,0.001


In [31]:
gan_model.optimizerG.param_groups[0]['lr']
metrics_df['lr_d']


0     0.001000
1     0.001000
2     0.001000
3     0.001000
4     0.001000
5     0.000250
6     0.000250
7     0.000250
8     0.000250
9     0.000250
10    0.000063
11    0.000063
12    0.000063
13    0.000063
14    0.000063
15    0.000016
16    0.000016
17    0.000016
18    0.000016
19    0.000016
20    0.000016
21    0.000016
22    0.000016
23    0.000016
24    0.000016
25    0.000016
26    0.000016
27    0.000016
28    0.000016
29    0.000016
30    0.000016
31    0.000016
32    0.000016
33    0.000016
34    0.000016
35    0.000016
36    0.000016
37    0.000016
38    0.000016
39    0.000016
40    0.000016
41    0.000016
42    0.000016
43    0.000016
44    0.000016
45    0.000016
46    0.000016
47    0.000016
48    0.000016
49    0.000016
50         NaN
Name: lr_d, dtype: float64

In [ ]:
# gdict

### Debug

In [ ]:
# class Generator(nn.Module):
#     def __init__(self, gdict):
#         super(Generator, self).__init__()

#         ## Define new variables from dict
#         keys=['ngpu','nz','nc','ngf','kernel_size','stride','g_padding']
#         ngpu, nz,nc,ngf,kernel_size,stride,g_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())

#         self.main = nn.Sequential(
#             # nn.ConvTranspose2d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
#             nn.Linear(nz,nc*ngf*8*8*8),# 32768
#             nn.BatchNorm2d(nc,eps=1e-05, momentum=0.9, affine=True),
#             nn.ReLU(inplace=True),
#             View(shape=[-1,ngf*8,8,8]),
#             nn.ConvTranspose2d(ngf * 8, ngf * 4, kernel_size, stride, g_padding, output_padding=1, bias=False),
#             nn.BatchNorm2d(ngf*4,eps=1e-05, momentum=0.9, affine=True),
#             nn.ReLU(inplace=True),
#             # state size. (ngf*4) x 8 x 8
#             nn.ConvTranspose2d( ngf * 4, ngf * 2, kernel_size, stride, g_padding, 1, bias=False),
#             nn.BatchNorm2d(ngf*2,eps=1e-05, momentum=0.9, affine=True),
#             nn.ReLU(inplace=True),
#             # state size. (ngf*2) x 16 x 16
#             nn.ConvTranspose2d( ngf * 2, ngf, kernel_size, stride, g_padding, 1, bias=False),
#             nn.BatchNorm2d(ngf,eps=1e-05, momentum=0.9, affine=True),
#             nn.ReLU(inplace=True),
#             # state size. (ngf) x 32 x 32
#             nn.ConvTranspose2d( ngf, nc, kernel_size, stride,g_padding, 1, bias=False),
#             nn.Tanh()
#         )
    
#     def forward(self, ip):
#         return self.main(ip)

# class Discriminator(nn.Module):
#     def __init__(self, gdict):
#         super(Discriminator, self).__init__()
        
#         ## Define new variables from dict
#         keys=['ngpu','nz','nc','ndf','kernel_size','stride','d_padding']
#         ngpu, nz,nc,ndf,kernel_size,stride,d_padding=list(collections.OrderedDict({key:gdict[key] for key in keys}).values())        

#         self.main = nn.Sequential(
#             # input is (nc) x 64 x 64
#             # nn.Conv2d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
#             nn.Conv2d(nc, ndf,kernel_size, stride, d_padding,  bias=True),
#             nn.BatchNorm2d(ndf,eps=1e-05, momentum=0.9, affine=True),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf) x 32 x 32
#             nn.Conv2d(ndf, ndf * 2, kernel_size, stride, d_padding, bias=True),
#             nn.BatchNorm2d(ndf * 2,eps=1e-05, momentum=0.9, affine=True),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*2) x 16 x 16
#             nn.Conv2d(ndf * 2, ndf * 4, kernel_size, stride, d_padding, bias=True),
#             nn.BatchNorm2d(ndf * 4,eps=1e-05, momentum=0.9, affine=True),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*4) x 8 x 8
#             nn.Conv2d(ndf * 4, ndf * 8, kernel_size, stride, d_padding, bias=True),
#             nn.BatchNorm2d(ndf * 8,eps=1e-05, momentum=0.9, affine=True),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*8) x 4 x 4
#             nn.Flatten(),
#             nn.Linear(nc*ndf*8*8*8, 1)
# #             nn.Sigmoid()
#         )

#     def forward(self, ip):
# #         print(ip.shape)
#         results=[ip]
#         lst_idx=[]
#         for i,submodel in enumerate(self.main.children()):
#             mid_output=submodel(results[-1])
#             results.append(mid_output)
#             ## Select indices in list corresponding to output of Conv layers
#             if submodel.__class__.__name__.startswith('Conv'):
# #                 print(submodel.__class__.__name__)
# #                 print(mid_output.shape)
#                 lst_idx.append(i)

#         FMloss=True
#         if FMloss:
#             ans=[results[1:][i] for i in lst_idx + [-1]]
#         else :
#             ans=results[-1]
#         return ans



In [ ]:
# netG = Generator(gdict).to(gdict['device'])
# netG.apply(weights_init)
# # # #     print(netG)
# # summary(netG,(1,1,64))
# # Create Discriminator
# netD = Discriminator(gdict).to(gdict['device'])
# netD.apply(weights_init)
# #     print(netD)
# summary(netD,(1,128,128))

In [ ]:
# noise = torch.randn(gdict['batchsize'], 1, 1, gdict['nz'], device=gdict['device'])
# fake = netG(noise)            
# # Forward pass real batch through D
# output = netD(fake)
# print([i.shape for i in output])

In [32]:
0.5**10

0.0009765625

In [33]:
70000/(8*6*8)

182.29166666666666

In [37]:
gdict.keys()

dict_keys(['ip_fname', 'op_loc', 'image_size', 'num_imgs', 'workers', 'nc', 'nz', 'ngf', 'ndf', 'beta1', 'kernel_size', 'stride', 'g_padding', 'd_padding', 'flip_prob', 'bns', 'checkpoint_size', 'batch_size', 'epochs', 'learn_rate', 'op_size', 'lr_epochs', 'lr_stepsize', 'deterministic', 'seed', 'lambda_spec_mean', 'lambda_spec_var', 'lambda_fm', 'lambda_gp', 'grad_clip', 'save_steps_list', 'run_suffix', 'description', 'config', 'mode', 'ip_fldr', 'facility', 'distributed', 'ngpu', 'device', 'multi-gpu', 'world_size', 'world_rank', 'local_rank', 'save_dir', 'best_chi1', 'best_chi2', 'iters', 'start_epoch'])

In [40]:
for key in ['batch_size','num_imgs','ngpu']:
    print(key,gdict[key])

batch_size 4
num_imgs 40
ngpu 1


In [49]:
gdict['world_size']

1